In [1]:
import pandas as pd
import numpy as np
import os
#from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.metrics import classification_report
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler ,normalize
from sklearn import preprocessing
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from collections import Counter
%matplotlib inline
mpl.style.use( 'ggplot' )
sns.set_style( 'white' )
pylab.rcParams[ 'figure.figsize' ] = 8 , 6


C:\Users\noni0\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = pd.read_csv('../../data/Data_set.csv',encoding='cp949')
df1 = pd.read_csv('../../data/Test_set.csv',encoding='cp949')
df1.AGE.value_counts()

50    382
45    374
55    322
40    309
35    214
60    191
30     92
65     68
25     53
70      5
20      5
*       4
Name: AGE, dtype: int64

In [4]:
def modefinder(numbers): #최빈값 찾는 함수
    c = Counter(numbers)
    mode = c.most_common(1)
    return mode[0][0]

In [5]:
def age_mean(columns): #나이에서 *인값을 제외한 평균
    for i in range(len(columns)):
        if (columns[i] == '*') == True:
            del columns[i]
    return np.mean(np.array(columns),dtype=np.float64 )

In [6]:
mean_age = age_mean(df.AGE)

In [7]:
def mode_sex(columns): #성별이'*'이면 최빈값 할당
    for i in range(len(columns)):
        if (columns[i] == '*') == True:
            columns[i] = modefinder(columns)
    return columns

In [8]:
df.SEX= mode_sex(df.SEX)

C:\Users\noni0\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [9]:
def mode_paym_metd(columns): #납부방법이 null이면 최빈값 할당
    null_column = df[columns.isnull() == True].index.tolist()
    for i in null_column:
            columns[i] = modefinder(columns)
    return columns

In [10]:
df.PAYM_METD = mode_paym_metd(df.PAYM_METD)

C:\Users\noni0\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [11]:
# plt.scatter(df["AGE"],df["LAST_CHLD_AGE"], color = "orange", s = 5)
# plt.xlabel("AGE")
# plt.ylabel("LAST_CHLD_AGE")
# plt.title("AGE-LASTCHLD")
# plt.legend()
# plt.show()

In [11]:
def age(columns): #자녀나이로 본인나이 측정, 자녀나이도 4세부터 5세단위로 증가함-> =만 써도 됨
    j = 15
    k = 4
    while j < 70:
        j = j+5
        k = k+5
        columns = np.where((columns == "*") & (k == df["LAST_CHLD_AGE"]), j+5 ,columns)
        columns = np.where((columns == "*") & (df["LAST_CHLD_AGE"] ==0), mean_age ,columns)
        columns = np.where((columns == "*") & (df["LAST_CHLD_AGE"] ==4), mean_age ,columns) 
        columns = np.where((columns == "*"),mean_age,columns)
        if j >70:
            break
    return columns

In [12]:
df.AGE = age(df.AGE)

In [13]:
df["AGE"] = df["AGE"].astype(float)
#성인나이로 막내자녀나이 추측하는 함수
#이상치 제거한 후(나이차이 열살이하로 나면 최빈값으로 바꿔줌),막내자녀나이가 0이 아닐 때 부모 나잇대별 평균막내자녀나이구한후
#부모나잇대에 맞춰서 평균막내자녀나이 할당
def mean_chld_age(columns):
    set_value_age = list(set(df["AGE"].tolist()))
    average_chld_age = []
    for value in set_value_age:
        average_chld = df["LAST_CHLD_AGE"].loc[((df["AGE"] == value) \
                             & ~(df["LAST_CHLD_AGE"] == 0.0)&(df["AGE"] - df["LAST_CHLD_AGE"]>= 20))].mean()
        average_chld_age.append(average_chld)
    null_index = df[columns.isnull() == True].index.tolist()
    
    for null in null_index:
        columns[null] = 'a' 
    
    average_chld_age.sort()
    set_value_age.sort()
    for set_value,average in zip(set_value_age,average_chld_age):
        columns = np.where((columns == 'a')&(df["AGE"] == set_value),average,columns)

    return columns

In [14]:
df["LAST_CHLD_AGE"] = mean_chld_age(df["LAST_CHLD_AGE"])

C:\Users\noni0\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\noni0\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


## MATE_OCCP_NAME_G * 제거

In [15]:
#주부의 추정 소득이 0이고 NaN의 추정소득이 0이기에 NaN값을 주부로 바꿔줌
#고소득 전문직 5848.75
df.MATE_OCCP_NAME_G = df["MATE_OCCP_NAME_G"].fillna("주부")
print(df.MATE_OCCP_NAME_G.value_counts())
of_index = df.MATE_OCCP_NAME_G[df.MATE_OCCP_NAME_G=="2차산업 종사자"].index.tolist()
df.MATE_JOB_INCM.loc[of_index].mean()

주부          57392
사무직         10053
2차산업 종사자     8107
자영업          6380
3차산업 종사자     4011
운전직          2887
공무원          2400
전문직          2105
1차산업 종사자     1411
단순 사무직       1316
*            1027
기업/단체 임원      931
고소득 전문직       560
단순 노무직        549
기타            534
예체능계 종사자      461
학생            109
Name: MATE_OCCP_NAME_G, dtype: int64


3812.359689157518

In [16]:
star_index = df[df['MATE_OCCP_NAME_G'] == "*"].index.tolist()

In [17]:
#직업이 *인 사람 중 추정 소득이 0인 사람 주부로 바꿔줌
star_data = df.MATE_JOB_INCM.loc[star_index]
zero_index = star_data[star_data==0].index.tolist()
df.MATE_OCCP_NAME_G.loc[zero_index] = "주부"
no_zero = star_data[star_data != 0]
#* 추정소득 4500이상인 사람 고소득 전문직으로 변경 
#* 추정소득 4500미만인 사람 
high_index = no_zero[no_zero >= 4500].index.tolist()
low_index = no_zero[no_zero < 4500].index.tolist()
df.MATE_OCCP_NAME_G.loc[high_index] = "고소득 전문직"
df.MATE_OCCP_NAME_G.loc[low_index] = "2차산업 종사자"

C:\Users\noni0\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


## OCCP_NAME_G *제거

In [18]:
print(df.OCCP_NAME_G.value_counts())
df.OCCP_NAME_G = df["OCCP_NAME_G"].fillna("주부")
s_index = df[df["OCCP_NAME_G"]=="*"].index.tolist()
s_data = df.CUST_JOB_INCM.loc[s_index]
zero_ix = s_data[s_data==0].index.tolist()
df.OCCP_NAME_G[zero_ix] = "주부"
#고소득 전문직, 단순 사무직
on_index = df.OCCP_NAME_G[df.OCCP_NAME_G=="단순 사무직"].index.tolist()
df.CUST_JOB_INCM.loc[on_index].mean()
h_index = s_data[s_data >= 4700].index.tolist()
l_index = s_data[s_data < 4700].index.tolist()
df.OCCP_NAME_G.loc[h_index] = "고소득 전문직"
df.OCCP_NAME_G.loc[l_index] = "단순 사무직"

주부          27565
사무직         16581
2차산업 종사자     9601
자영업          9485
3차산업 종사자     8275
공무원          5091
전문직          5043
단순 사무직       4107
학생           3835
운전직          2126
기타           1672
고소득 전문직      1223
*            1189
1차산업 종사자     1178
기업/단체 임원     1041
예체능계 종사자      936
단순 노무직        821
Name: OCCP_NAME_G, dtype: int64


C:\Users\noni0\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\noni0\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [19]:
np.where((df["TEL_CNTT_QTR"]==20162)&(df["TEL_MBSP_GRAD"].isnull() != True))

(array([ 2291,  2534,  4753,  4839,  5066,  6637,  7049,  7500,  8812,
         9084,  9086,  9470,  9664, 10006, 10274, 10528, 11698, 11760,
        12581, 12710, 12876, 13089, 13133, 16297, 16367, 16437, 19334,
        19416, 21083, 21634, 21714, 22952, 23585, 25391, 25613, 25798,
        26040, 28584, 29484, 29527, 30961, 31339, 31382, 32257, 32481,
        32789, 32871, 32965, 33226, 33340, 33508, 37429, 39639, 39864,
        40951, 41275, 41490, 42002, 42206, 42676, 42980, 43391, 43514,
        43556, 44075, 44109, 44237, 44325, 44383, 44659, 44816, 45044,
        45358, 45414, 46487, 47264, 49001, 50160, 50188, 50421, 50558,
        50958, 51391, 51777, 52356, 53610, 55329, 55342, 55988, 56833,
        56919, 57655, 57860, 58882, 58889, 59613, 59772, 61146, 61914,
        63007, 63696, 65371, 65987, 66301, 66320, 66690, 66762, 66768,
        66777, 66943, 67166, 67272, 67645, 68304, 68699, 69636, 69981,
        71075, 72267, 72825, 73090, 73324, 74241, 74526, 75717, 76417,
      

In [20]:
df.TEL_MBSP_GRAD = df.TEL_MBSP_GRAD.fillna('Q')

In [21]:
df.isnull().sum()

CUST_ID              0
TARGET               0
BNK_LNIF_CNT         0
CPT_LNIF_CNT         0
SPART_LNIF_CNT       0
ECT_LNIF_CNT         0
TOT_LNIF_AMT         0
TOT_CLIF_AMT         0
BNK_LNIF_AMT         0
CPT_LNIF_AMT         0
CRDT_OCCR_MDIF       0
SPTCT_OCCR_MDIF      0
CRDT_CARD_CNT        0
CTCD_OCCR_MDIF       0
CB_GUIF_CNT          0
CB_GUIF_AMT          0
OCCP_NAME_G          0
CUST_JOB_INCM        0
HSHD_INFR_INCM       0
ACTL_FMLY_NUM        0
CUST_FMLY_NUM        0
LAST_CHLD_AGE        0
MATE_OCCP_NAME_G     0
MATE_JOB_INCM        0
CRDT_LOAN_CNT        0
MIN_CNTT_DATE        0
TOT_CRLN_AMT         0
TOT_REPY_AMT         0
CRLN_OVDU_RATE       0
CRLN_30OVDU_RATE     0
                    ..
GDINS_MON_PREM       0
SVINS_MON_PREM       0
FMLY_GDINS_MNPREM    0
FMLY_SVINS_MNPREM    0
MAX_MON_PREM         0
TOT_PREM             0
FMLY_TOT_PREM        0
CNTT_LAMT_CNT        0
LT1Y_CTLT_CNT        0
AUTR_FAIL_MCNT       0
FYCM_PAID_AMT        0
FMLY_CLAM_CNT        0
FMLY_PLPY_C

In [22]:
del df['CUST_ID']
df = pd.get_dummies(df)

In [23]:
df_test = df[:2000]
df_train = df[2000:]

In [24]:
x_data = df_train.drop('TARGET',axis=1)
y_data = df_train.TARGET

In [25]:

X = x_data
y = y_data
from imblearn.over_sampling import ADASYN 
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import CondensedNearestNeighbour 

# undersampling
# cnn = CondensedNearestNeighbour(random_state=42) 
# X_resampled, y_resampled = cnn.fit_sample(X, y)

#RANDOM 
# ros = RandomOverSampler(random_state=42)
# X_resampled, y_resampled = ros.fit_sample(X, y)

#ADASYN
# ada = ADASYN(random_state=42,n_neighbors=3)
# X_resampled, y_resampled = ada.fit_sample(X, y)

# Apply SMOTE + ENN
sm = SMOTE(kind='borderline2')
X_resampled, y_resampled = sm.fit_sample(X, y)

In [26]:
df_1 = pd.DataFrame(X_resampled)
df_2 = pd.DataFrame(y_resampled)
df_2.columns = ['TARGET']
df_final = pd.concat([df_1,df_2],axis=1)
df_final.head()

,0,1,2,3,4,5,6,7,8,9,...,126,127,128,129,130,131,132,133,134,TARGET
0,0.0,0.0,1.0,1.0,48001.0,48001.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0
1,2.0,0.0,0.0,0.0,48001.0,1.0,48001.0,0.0,13.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0
2,1.0,2.0,2.0,0.0,15001.0,3001.0,3001.0,12001.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1
3,1.0,0.0,0.0,0.0,9001.0,9001.0,9001.0,0.0,25.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0
4,2.0,0.0,0.0,0.0,39001.0,39001.0,39001.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0


In [27]:
df_final = df_final.sample(frac=1)
df_final.head()

,0,1,2,3,4,5,6,7,8,9,...,126,127,128,129,130,131,132,133,134,TARGET
84845,2.000000,2.000000,4.000000,2.000000,21001.000000,12001.000000,12001.000000,3001.000000,1.000000,61.000000,...,0.000000,1.000000,0.000000,1.0,0.0,0.000000,0.000000,1.0,0.0,0
130579,0.000000,1.685342,2.685342,1.000000,7585.038152,5057.025435,0.000000,7584.880823,16.775898,26.887949,...,0.157329,1.000000,0.000000,0.0,0.0,0.842671,0.157329,0.0,1.0,1
63330,0.000000,2.000000,2.000000,0.000000,12001.000000,0.000000,0.000000,12001.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.0,1.0,0.000000,0.000000,0.0,1.0,0
35973,0.000000,1.000000,2.000000,1.000000,141001.000000,141001.000000,0.000000,3001.000000,13.000000,37.000000,...,0.000000,1.000000,0.000000,1.0,0.0,0.000000,0.000000,0.0,1.0,0
150204,0.359625,1.359625,1.719249,0.359625,72474.244718,68158.748239,7552.478463,45502.391725,31.738014,49.000000,...,0.000000,0.359625,0.640375,0.0,1.0,0.000000,0.000000,0.0,1.0,1


In [28]:
train_y = df_final.TARGET
train_x = df_final.drop('TARGET',axis=1)

In [29]:
test_y = df_test.TARGET
test_x = df_test.drop('TARGET',axis=1)

In [30]:
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0,5))
train_x = min_max_scaler.fit_transform(train_x)
test_x = min_max_scaler.fit_transform(test_x)

In [31]:
from sklearn import linear_model
reg = linear_model.LogisticRegression()
reg.fit(train_x,train_y)
score = reg.score(test_x,test_y)
print('Logistic Regression Accuracy : ',score)

Logistic Regression Accuracy :  0.832


In [32]:
from sklearn.metrics import classification_report
print(classification_report(test_y,reg.predict(test_x)))

             precision    recall  f1-score   support

          0       0.99      0.83      0.91      1920
          1       0.16      0.78      0.27        80

avg / total       0.96      0.83      0.88      2000



In [33]:
reg2 = linear_model.LogisticRegression(class_weight='balanced')
reg2.fit(train_x,train_y)
score2 = reg2.score(test_x,test_y)
print('Logistic Regression used class weight ' ,score2)
print(classification_report(test_y,reg2.predict(test_x)))

Logistic Regression used class weight  0.832
             precision    recall  f1-score   support

          0       0.99      0.83      0.91      1920
          1       0.16      0.78      0.27        80

avg / total       0.96      0.83      0.88      2000



In [34]:
# fit model no training data
model = XGBClassifier(learning_rate=0.2,max_depth=7,base_score=0.5,max_delta_step=0,n_estimators=100)

pipe = Pipeline([['xg',MinMaxScaler(feature_range=(0,5))],['model',model]])
pipe.fit(train_x,train_y)

print(classification_report(test_y,pipe.predict(test_x)))
print("ACCURACY:",pipe.score(test_x,test_y))

             precision    recall  f1-score   support

          0       0.98      0.96      0.97      1920
          1       0.35      0.46      0.40        80

avg / total       0.95      0.94      0.95      2000

ACCURACY: 0.9435


AGE GAP 이 15 : 
1.Logistic Regression Accuracy :  0.831

2.classification_report
             precision    recall  f1-score   support

          0       0.99      0.83      0.90      1920
          1       0.16      0.79      0.27        80

avg / total       0.96      0.83      0.88      2000

3.Logistic Regression used class weight  0.831
             precision    recall  f1-score   support

          0       0.99      0.83      0.90      1920
          1       0.16      0.79      0.27        80

avg / total       0.96      0.83      0.88      2000

4.XGB

                precision    recall  f1-score   support

          0       0.97      0.97      0.97      1920
          1       0.31      0.38      0.34        80

avg / total       0.95      0.94      0.94      2000

ACCURACY: 0.942


AGE GAP이 10:
    
1.Logistic Regression Accuracy :  0.831

2.classification_report
             precision    recall  f1-score   support

          0       0.99      0.83      0.90      1920
          1       0.16      0.79      0.27        80

avg / total       0.96      0.83      0.88      2000

3.Logistic Regression used class weight  0.831
             precision    recall  f1-score   support

          0       0.99      0.83      0.90      1920
          1       0.16      0.79      0.27        80

avg / total       0.96      0.83      0.88      2000


4.             precision    recall  f1-score   support

          0       0.98      0.96      0.97      1920
          1       0.37      0.53      0.43        80

avg / total       0.96      0.94      0.95      2000

ACCURACY: 0.9445


# 결론 
F1-score기준
Logistic : 20 > 10 == 15 (0.001차이)
classification: 10==15==20
logistic+ weight:  10==15==20
XGB: 10> 20==15 (3차이)
    